<a href="https://colab.research.google.com/github/Karensgh/MURA/blob/master/Model_Serving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#select GPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow.keras as keras
import sys
sys.modules['keras'] = keras
from keras.applications.vgg16 import VGG16
from keras.layers import Input,Flatten,Dense,Dropout,GlobalAveragePooling2D
from keras.models import Sequential,Model
from keras import optimizers

## Model Set up

In [3]:
base = VGG16(weights ='imagenet',input_shape=(128, 128, 3),include_top=False)
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid', name='predictions')(x)
# define new model
vgg = Model(inputs=base.input, outputs=predictions)
vgg.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
______________________________________________________________

In [5]:
from google.colab import files
uploaded_data=files.upload()

Saving VGG_unfreeze_b5.weights.05-0.48.hdf5 to VGG_unfreeze_b5.weights.05-0.48.hdf5


In [6]:
vgg.load_weights('VGG_unfreeze_b5.weights.05-0.48.hdf5')
vgg.save('vgg_SavedModel.hdf5', save_format='tf')

In [7]:
vgg_model = keras.models.load_model('vgg_SavedModel.hdf5')

In [8]:
vgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [9]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile
import tensorflow as tf
""
MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    vgg,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /tmp/1/assets

Saved model:
total 320
drwxr-xr-x 2 root root   4096 Jun 22 21:37 assets
-rw-r--r-- 1 root root 315707 Jun 22 21:37 saved_model.pb
drwxr-xr-x 2 root root   4096 Jun 22 21:37 variables


In [10]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 128, 128, 3)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['predictions'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0622 21:37:20.917082 140327935469440 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/o

In [11]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   6812      0 --:--:-- --:--:-- --:--:--  6796
OK
Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,622 B]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://storag

## Install Tensorflow Serving

In [12]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 53 not upgraded.
Need to get 187 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.2.0 [187 MB]
Fetched 187 MB in 3s (56.0 MB/s)
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.2.0_all.deb ...
Unpacking tensorflow-model-server (2.2.0) ...
Setting up tensorflow-model-server (2.2.0) ...


In [13]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [14]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=MURA_Prediction \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


Starting job # 0 in a separate thread.


In [15]:
!tail server.log

2020-06-22 21:38:00.356185: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-06-22 21:38:00.422186: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2020-06-22 21:38:00.711919: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:183] Running initialization op on SavedModel bundle at path: /tmp/1
2020-06-22 21:38:00.734186: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:364] SavedModel load for tags { serve }; Status: success: OK. Took 392663 microseconds.
2020-06-22 21:38:00.736887: I tensorflow_serving/servables/tensorflow/saved_model_warmup.cc:105] No warmup data file found at /tmp/1/assets.extra/tf_serving_warmup_requests
2020-06-22 21:38:00.737066: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: MURA_Prediction version: 1}
2020-06-22 21

## Make a request to the model

In [79]:
## validation image
from google.colab import files
test_images=files.upload()

Saving image1.png to image1.png
Saving image2.png to image2.png
Saving image3.png to image3.png


In [106]:
validation_dir='/content/test/images'

In [110]:
from keras_preprocessing.image import ImageDataGenerator
idg_valid_settings = dict(samplewise_center = True,
                         samplewise_std_normalization = True,
                          rotation_range = 0, 
                          width_shift_range = 0., 
                         height_shift_range = 0.,
                         zoom_range = 0.0, 
                         horizontal_flip = False,
                         vertical_flip = False)
valid_gen = ImageDataGenerator(**idg_valid_settings)

In [111]:
validation_generator = valid_gen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=64,
        class_mode='binary',shuffle=False)
print(validation_generator.class_indices)

Found 3 images belonging to 3 classes.
{'.ipynb_checkpoints': 0, 'negative': 1, 'positive': 2}


In [115]:
test_images=next(validation_generator)[0]
test_images.shape

(3, 128, 128, 3)

In [116]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... 423, -0.43438276648521423, -0.43438276648521423]]]]}


In [117]:
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/MURA_Prediction:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']


In [118]:
class_names=['negative','positive']
test_labels=['positive','positive','positive']
images=['image1.png','image2.png','image3.png']

In [122]:
from PIL import Image
for i in range(0,3):
  print('Precdict: {} (probability {}) \nActual: {}'.format(
    class_names[(predictions[i][0]> 0.4)*1], predictions[i], test_labels[i])) 
  print()                                       

Precdict: positive (probability [0.650100291]) 
Actual: positive

Precdict: positive (probability [0.63672322]) 
Actual: positive

Precdict: positive (probability [0.594615877]) 
Actual: positive

